<a href="https://colab.research.google.com/github/tramyynt/Techlabs/blob/master/Pickshare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
import pandas as pd
import json
import os
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

from google.colab import drive

In [3]:
drive.mount('/drive')

Mounted at /drive


In [4]:
users = pd.read_json('/drive/MyDrive/Techlabs/pickshare_users.json')
# users.head(5)
packages = pd.read_json('/drive/MyDrive/Techlabs/pickshare_packages.json')
packages.head(5)

,senderCompanyName,city,_owner
0,Amazon,Dortmund,Björn
1,Zoo Royal,Berlin,Pia
2,ABOUTYOU,Dortmund,Björn
3,Test,Berlin,VTest
4,notebooksbilliger.de,Dortmund,Björn


In [4]:
print(packages['_owner'].describe())
print(packages['city'].describe())
print(users['firstName'].describe())
print(users['city'].describe())

count        3408
unique        100
top       Karsten
freq          252
Name: _owner, dtype: object
count       3362
unique        39
top       Berlin
freq        2345
Name: city, dtype: object
count      257
unique     209
top       Test
freq         5
Name: firstName, dtype: object
count        257
unique        61
top       Berlin
freq         102
Name: city, dtype: object


In [5]:
print(packages['senderCompanyName'].describe())

count       3408
unique      1002
top       Amazon
freq        1055
Name: senderCompanyName, dtype: object


1. Cluster geospatial data to know where we should put a depot
2. Cluster based on their total numbers of oders.

### Data Preprocessing

In [72]:
def  cleaning_text(df, col):
  df.loc[df[col].str.contains('berlin', case=False, regex=False), col] = 'Berlin'
  df.loc[df[col].str.contains('dortmund', case=False, regex=False), col] = 'Dortmund'
  df.loc[df[col].str.contains('Potsdam', case=False, regex=False), col] = 'Potsdam'
  df.loc[df[col].str.contains('Dusseldorf', case=False, regex=False), col] = 'Düsseldorf'
  df.loc[df[col].str.contains('Regensburg', case=False, regex=False), col] = 'Regensburg'
  df.loc[df[col].str.contains('Moenchengladbach', case = False, regex = False), col] = 'Mönchengladbach'
  df.loc[df[col].str.contains('Bad Hersfeld', case = False, regex = False), col] = 'Bad Hersfeld'
  df.loc[df[col].str.contains('Viersen', case = False, regex = False), col] = 'Viersen'
  df.loc[df[col].str.contains('Münster', case = False, regex = False), col] = 'Münster'
  df.loc[df[col].str.contains('mönchengladbach', case = False, regex = False), col] = 'Mönchengladbach'
  df.loc[df[col].str.contains('Wetzlar', case = False, regex = False), col] = 'Wetzlar'
  df.loc[df[col].str.contains('München', case = False, regex = False), col] = 'München'
  return df

In [62]:
#packages
#packages.isna().sum()
packages = packages[packages.city.notna()].reset_index(drop=True) #remove missing values

#cleaning text
packages = cleaning_text(packages, 'city')

In [73]:
#users
users = users[users.city.notna()].reset_index(drop = True)

users = cleaning_text(users,'city')

In [111]:
#get city list of Germany and check if a city belongs to the list, otherwise remove it. Refer : https://moduliertersingvogel.de/2017/09/03/german-cities-list/
from bs4 import BeautifulSoup
import requests
class CityList:
    def __init__(self):
        self.__countries={
            'BY':'Bayern',
            'BW':'Baden-Württemberg',
            'NW':'Nordrhein-Westfalen',
            'HE':'Hessen',
            'SN':'Sachsen',
            'NI':'Niedersachsen',
            'RP':'Rheinland-Pfalz',
            'TH':'Thüringen',
            'BB':'Brandenburg',
            'ST':'Sachsen-Anhalt',
            'MV':'Mecklenburg-Vorpommern',
            'SH':'Schleswig-Holstein',
            'SL':'Saarland',
            'HB':'Bremen',
            'BE':'Berlin',
            'HH':'Hamburg'
        }
        
    def retrieveGermanList(self):
        r = requests.get('https://de.wikipedia.org/wiki/Liste_der_St%C3%A4dte_in_Deutschland')
        soup = BeautifulSoup(r.content, "html5lib")
        cities={}
        tables=soup.find_all('table')
        for t in tables:
            lis=t.find_all('dd')
            for l in lis:
                # All countries are in brackets after the city name.
                # Some cities are listed like: SN, Landeshauptstadt
                countryShort=None
                additional=l.contents[1].split('(')[1].split(')')[0].strip()
                if ',' in additional:
                    countryShort=additional.split(',')[0]
                else:
                    countryShort=additional
                cities[l.find('a').contents[0]]=self.__countries[countryShort]
                
        return cities

city_list = CityList().retrieveGermanList()
users = users[users['city'].isin(city_list.keys())]
packages = packages[packages['city'].isin(city_list.keys())]

In [109]:
users_per_city = users['city'].value_counts().to_frame().reset_index(drop = False)
users_per_city.columns = ['city', 'number_users']
users_per_city

,city,number_users
0,Berlin,115
1,Dortmund,36
2,Mönchengladbach,24
3,Wetzlar,12
4,Münster,10
5,Potsdam,5
6,Bad Hersfeld,4
7,Düsseldorf,3
8,Regensburg,3
9,Viersen,2


In [110]:
packages_per_city = packages['city'].value_counts().to_frame().reset_index(drop=False)
packages_per_city.columns = ['city','num_packages']
packages_per_city

,city,num_packages
0,Berlin,2768
1,Dortmund,306
2,Münster,50
3,Regensburg,38
4,Wetzlar,36
5,Potsdam,22
6,Düsseldorf,18
7,Mönchengladbach,9
8,Grevenbroich,5
9,Grünberg,4


## Exploratory Data Analysis
1. Number of packages per city 
2. Number of users per city
3. Number of packages per sender